In [1]:
# imports
from IPython.display import clear_output
from PIL import Image
import os
import json
# clear
clear_output(wait=True)
# installing kaggle
!pip install -q kaggle
# user and key info
info = {"username":"jackbellamy","key":"3442898ba5e904ac3a48eb5b59398805"}
# opening file
json_object = json.dumps(info, indent=4)
with open("kaggle.json", "w") as outfile:
    outfile.write(json_object)
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
# copying data to content file of kaggle which can be found on the left hand side of the screen
if not (os.path.exists("/content/nfl-big-data-bowl-2022")):
  !kaggle competitions download -c nfl-big-data-bowl-2022
  !mkdir nfl-big-data-bowl-2022
  !unzip content/nfl-big-data-bowl-2022.zip -d nfl-big-data-bowl-2022

^C
ERROR: Operation cancelled by user

[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
mkdir: /Users/ulric/.kaggle: File exists
nfl-big-data-bowl-2022.zip: Skipping, found more recently modified local copy (use --force to force download)
unzip:  cannot find or open content/nfl-big-data-bowl-2022.zip, content/nfl-big-data-bowl-2022.zip.zip or content/nfl-big-data-bowl-2022.zip.ZIP.


In [3]:
import pandas as pd
# reading the data to variables
scouting = pd.read_csv("content/nfl-big-data-bowl-2022/PFFScoutingData.csv")
games = pd.read_csv("content/nfl-big-data-bowl-2022/games.csv")
players = pd.read_csv("content/nfl-big-data-bowl-2022/players.csv")
plays = pd.read_csv("content/nfl-big-data-bowl-2022/plays.csv")
tracking2018 = pd.read_csv("content/nfl-big-data-bowl-2022/tracking2018.csv")
tracking2019 = pd.read_csv("content/nfl-big-data-bowl-2022/tracking2019.csv")
tracking2020 = pd.read_csv("content/nfl-big-data-bowl-2022/tracking2020.csv")

In [4]:
def abbrvs(csv,column,abbrv,fullname):
  # new names
  type_name = []
  # looping through
  for i in csv[column]:
    # filtering out NaN values
    if i in abbrv:
      index = abbrv.index(i)
      type_name.append(fullname[index])
    else:
      type_name.append(i)
  return type_name


def team_number_splitting(csv,column,symbol):
  # splitting to team and number
  split_team = []
  split_numbers = []
  # splitting the data
  split = csv[column].str.split(symbol, expand = True)
  # loop for each
  for i in range(0,len(csv)):
    # specific data
    split.iloc[i]
    # number of NaN
    nan_num = split.iloc[i].isna().sum()
    max = len(split.iloc[i])
    if max-nan_num == 0:
      split_team.append(csv[column][i])
      split_numbers.append(csv[column][i])
    else:
      info = []
      for j in range(0,(max-nan_num)):
        split_i = split[j][i].split(" ")
        info.append(split_i)
      # team name
      split_team.append(info[0][0])
      # numbers
      nums = []
      for k in range(0,max-nan_num):
        num = info[k][1]
        nums.append(num)
      split_numbers.append(nums)
  return split_team,split_numbers


def number_splitting(csv,column,symbol,n):
  split = csv[column].str.split(symbol, n = n, expand = True)
  return split


def pd_to_num(csv,split,column_names):
  split.columns = column_names
  # adding columns
  for name in column_names:
    csv[name] = pd.to_numeric(split[name])
  return

In [5]:
# abbreviations
# 'kickType'
csv = scouting
column = "kickType"
kick_types = ['D','F','K','O','P','Q','S','B','N','R','A']
kick_type_names = ['Deep','Flat','Free','Obvious','Pooch','Squib','Surprise','Deep','Normal','Rugby','Aussie']
# calling function
kick_type_fullnames = abbrvs(csv,column,kick_types,kick_type_names)
# adding column
scouting["kick_type_name"] = kick_type_fullnames

# 'kickDirectionIntended', 'kickDirectionActual', 'returnDirectIntended', 'returnDirectionActual', 'KickTypeContact'

In [6]:
# team and numbers
# 'missedTackler'
csv = scouting

column = "missedTackler"
symbol = "; "
# calling function
missed_tackler_team, missed_tackler_numbers = team_number_splitting(csv,column,symbol)
# adding
scouting["missed_tackler_team"] = missed_tackler_team
scouting["missed_tackler_numbers"] = missed_tackler_numbers

# 'specialTeamsSafeties'
column = "specialTeamsSafeties"
symbol = "; "
# calling function
specialTeamsSafeties_team, specialTeamsSafeties_numbers = team_number_splitting(csv,column,symbol)
# adding
scouting["special_teams_safeties_team"] = specialTeamsSafeties_team
scouting["special_teams_safeties_numbers"] = specialTeamsSafeties_numbers

# 'Vises'
column = "vises"
symbol = "; "
# calling function
vises_team, vises_numbers = team_number_splitting(csv,column,symbol)
# adding
scouting["vises_team"] = vises_team
scouting["vises_numbers"] = vises_numbers

In [7]:
# number splitting
# 'kickoffReturnFormation'
csv = scouting
column = "kickoffReturnFormation"
symbol = "-"
n = 2
column_names = ['formation_front','formation_mid','formation_back']
# calling function
formation_split = number_splitting(csv,column,symbol,n)
# calling function
pd_to_num(csv,formation_split,column_names)

In [8]:
# number splitting
# 'gameDate'
csv = games

column = "gameDate"
symbol = "/"
n = 2
column_names = ['game_date_day','game_date_month','game_date_year']
# calling function
game_date_split = number_splitting(csv,column,symbol,n)
# calling function
pd_to_num(csv,game_date_split,column_names)

# 'gameTimeEastern'
column = "gameTimeEastern"
symbol = ":"
n = 2
column_names = ['game_time_hour','game_time_minute','game_time_second']
# calling function
game_time_split = number_splitting(csv,column,symbol,n)
# calling function
pd_to_num(csv,game_time_split,column_names)

In [9]:
# 'birthDate'
# splitting the numbers
players['birth_year'] = pd.DatetimeIndex(players['birthDate']).year



In [10]:
# 'height'
height_split = players["height"].str.split("-", n = 1, expand = True)
# new column names
height_split.columns = ['Feet', 'Inch']
# adding new columns
players["height_feet"] = pd.to_numeric(height_split["Feet"])
players["height_inch"] = pd.to_numeric(height_split["Inch"])
# still have 73 feet
# getting rid of NaN values
players["height_inch"] = players["height_inch"].fillna(0)
# replacing heights that did not have '-' at start
for i in players["height_feet"]:
    if i > 7:
        j = 0.0833333 * i
        players["height_feet"].replace({i : j}, inplace=True)
# inches to numerical value
players["height_inch"] = 0.0833333 * players["height_inch"]
# adding feet and inches to get one value
height_numerical = players["height_feet"] + players["height_inch"]
# adding new numerical value column
players["height_numerical"] = pd.to_numeric(height_numerical)

In [11]:
# number splitting
# 'gameClock'
csv = plays

column = "gameClock"
symbol = ":"
n = 2
column_names = ['game_clock_minute','game_clock_second','game_clock_milli']
# calling function
game_time_split = number_splitting(csv,column,symbol,n)
# calling function
pd_to_num(csv,game_time_split,column_names)

In [12]:
# x and y positions of players
x = tracking2018['x']
y = tracking2018['y']

# getting specific play and player, x and y
nfl_id = 39470
display_name = "Justin Tucker"

x_nfl_id_name = tracking2018.loc[((tracking2018["nflId"] == nfl_id) & (tracking2018["displayName"] == display_name)), "x"]
y_nfl_id_name = tracking2018.loc[((tracking2018["nflId"] == nfl_id) & (tracking2018["displayName"] == display_name)), "y"]

x_nfl_id_name.head()

0    41.32
1    41.68
2    42.05
3    42.43
4    42.84
Name: x, dtype: float64

# Xiaojun
## count plot

In [13]:
import plotly.graph_objects as go
import plotly.express as px
# from skimage import io

import datetime
# plays['year'] = plays['gameId'].apply(lambda x: int(str(x)[:4]))


data = plays.merge(games, on='gameId', how='left').groupby(['specialTeamsPlayType','season']).nunique().reset_index()[['specialTeamsPlayType','season','playId']]

fig = px.bar(
    data_frame=data,
    color="specialTeamsPlayType",
    y="playId",
    x="season",
    barmode="group",
    labels={'specialTeamsPlayType':'Type of Special team plays',
                   'playId':'Number of plays'},
             width=600,
             height=400
)

fig.write_html('count_figure.html', auto_open=True)

## scouting

D: Deep - your normal deep kick with decent hang time

F: Flat - different than a Squib in that it will have some hang time and no roll but has a lower trajectory and hang time than a Deep kick off

K: Free Kick - Kick after a safety

O: Obvious Onside - score and situation dictates the need to regain possession. Also the hands team is on for the returning team

P: Pooch kick - high for hangtime but not a lot of distance - usually targeting an upman

Q: Squib - low-line drive kick that bounces or rolls considerably, with virtually no hang time

S: Surprise Onside - accounting for score and situation an onsides kick that the returning team doesn’t expect. Hands teams probably aren't on the field

B: Deep Direct OOB - Kickoff that is aimed deep (regular kickoff) that goes OOB directly (doesn't bounce)

Possible values for punt plays:

N: Normal - standard punt style

R: Rugby style punt

A: Nose down or Aussie-style punts
                                                                                          
                                                                                          
                                                                

In [14]:

data = scouting.groupby('kickType').count().reset_index()[['kickType','playId']].sort_values(by='playId', ascending=False)

kick_dict = {
    "D": "Deep",
    "F": " Flat",
    "K": "Free kick",
    "O": "Obvious onside",
    "P": "Pooch kick",
    "Q": "Squib",
    "S": "Surprise",
    "B": "Deep Direct OOB",
    "N": "Normal",
    "R": "Rugby",
    "A": "Aussie"
}              

data['kickType'] = data['kickType'].apply(lambda x:kick_dict[x])


fig = px.bar(
    data_frame=data,
    y="playId",
    x="kickType",
    barmode="group",
    labels={'kickType':'Type of Special team plays',
                   'playId':'Number of plays'},
             width=600,
             height=400
)

fig.write_html('scouting_figure.html', auto_open=True)

## explore

In [15]:
cols = [ 'down', 'playResult']
data =plays[plays.down > 0][cols]  # ignore the kick-off

fig = px.box(data, x="down", y="playResult", width=600,height=400, labels={'playResult':'result yards'})
fig.write_html('explore_figure.html', auto_open=True)

# game

x: Player position along the long axis of the field, 0 - 120 yards. See Figure 1 below. (numeric)


y: Player position along the short axis of the field, 0 - 53.3 yards. See Figure 1 below. (numeric)

In [21]:
import plotly.graph_objects as go

marker_color = {
    "home": "red",
    "away": " blue",
    "football": "yellow"
}
# too large to complie 
# df = tracking2018[tracking2018.frameId == 1] 

# use one game to show
df = tracking2018[(tracking2018.frameId == 1)   & (tracking2018.gameId.isin([2018123000]))]

df['color'] = df.team.apply(lambda x:marker_color[x])

df = df.reset_index()


# Create figure
fig = go.Figure(
        
)

# Constants
img_width = 120
img_height = 54
scale_factor = 1


# Add image
import base64
polar_light = "pitch.jpg"
with open(polar_light, "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode()
#add the prefix that plotly will want when using the string as source
encoded_image = "data:image/png;base64," + encoded_string


fig.add_layout_image(
    dict(
        x=0,
        sizex=img_width * scale_factor,
        y=img_height * scale_factor,
        sizey=img_height * scale_factor,
        xref="x",
        yref="y",
        opacity=1.0,
        layer="below",
        sizing="stretch",
        source=encoded_image)
)

# Configure axes
fig.update_xaxes(
    visible=False,
    range=[0, img_width * scale_factor]
)

fig.update_yaxes(
    visible=False,
    range=[0, img_height * scale_factor],
    
    scaleanchor="x"
)



base = 23
names = []
for i in range(0,len(df), base):
    positions = df.iloc[i:i+base]
    name = df.gameId.values[0] + df.playId.values[0]
    names.append(name)
    fig.add_trace(
            go.Scatter(
                visible=False,
                x=positions.x,
                y=positions.y,
                marker_size=20,
                mode='markers',
    marker=dict(color=positions.color)
            ),

    )

    
    
    
fig.data[0].visible = True
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "current gameId & playId: " + str(names[i])}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"prefix": "current : "},
    steps=steps
)]


fig.update_layout(
    sliders=sliders,
    
)

fig.write_html('game.html', auto_open=True)

<ipython-input-21-673640f3bf9e>:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

